# A Quick Introduction to Model Context Protocol (MCP) in Python
This notebook is a complementary resource to Medium blog [A Quick Introduction to Model Context Protocol (MCP) in Python](https://medium.com/p/bee6d36334ec). 

In this notebook we'll show how LLMs struggle with simple tasks like counting letters, and how one can implement Function Calling to overcome such obsticals. Further on, the Function Call is added to a MCP server (sdir) with FastMCP which an Agent can access.

### ToC:
#### Can LLMs count how many R's are there in the word 'Strawberry'?
#### 1. Create an Agent with Funciton Call
#### 2. Create MCP server

## Imports

In [1]:
import os

import anthropic
from dotenv import load_dotenv

## Can LLMs count how many R's are there in the word 'Strawberry'?

In [2]:
load_dotenv()
API_KEY = os.getenv("ANTHROPIC_API_KEY")

user_prompt = "How many R's are there in the word 'Strawberry'?"

client = anthropic.Anthropic(api_key=API_KEY)


response = client.messages.create(
    model="claude-3-5-haiku-20241022",
    max_tokens=1024,
    messages=[{"role": "user", "content": user_prompt}]
)

In [5]:
print(response.content[0].text)

Let me count the R's in the word 'Strawberry':

S t r a w b e r r y

There are 2 R's in the word 'Strawberry'.


 ## 1. Create an Agent with Funciton Call

### Create Python Function

In [2]:
def count_letters(word:str, letter:str) -> int:
    """ Count letter in a word. """
    return word.lower().count(letter.lower())

### Create Agent

In [3]:
import os

import anthropic
from dotenv import load_dotenv

def query_agent(user_prompt:str) -> str:
    """ Agent abple to execute 'count_letters' function. """
    load_dotenv()
    API_KEY = os.getenv("ANTHROPIC_API_KEY")

    # Initialize the Anthropic client
    client = anthropic.Anthropic(api_key=API_KEY)

    # Define the tools that the agent can use
    tools = [
        {
            "name": "count_letters",
            "description": "Count letter in a word.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "word": {
                        "type": "string",
                        "description": "The word to count letters in."
                    },
                    "letter": {
                        "type": "string",
                        "description": "The letter to count in the word."
                    }
                },
                "required": ["word", "letter"]
            }
        }
    ]

    # Create a message to send to the agent
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        tools=tools,
        messages=[{"role": "user", "content": user_prompt}]
    )

    # Check if the agent's response includes a function call
    func_call = response.content[1].model_dump()

    if func_call["name"] == "count_letters":
        word = func_call["input"]["word"]
        letter = func_call["input"]["letter"]
        count = count_letters(word, letter)
        return f"There are {count} '{letter}'s in the word '{word}'."
    else:
        return "I count letters. Please provide a valid request."

### Run Agent

In [4]:
query_agent(user_prompt="How many R's are there in the word 'Strawberry'?")

"There are 3 'r's in the word 'Strawberry'."

## 2. Create MCP server
GiT - MCP Client: https://github.com/modelcontextprotocol/python-sdk?tab=readme-ov-file#adding-mcp-to-your-python-project  
SmoleAgents MCP Client: https://huggingface.co/docs/smolagents/en/tutorials/tools#tool-collection-from-a-collection-in-the-hub


In [ ]:
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

async def query_mcp_agent(user_prompt):
    """ Agent able to execute 'count_letters' function using MCP server. """
    load_dotenv()
    API_KEY = os.getenv("ANTHROPIC_API_KEY")

    # Create server parameters for stdio connection
    server_params = StdioServerParameters(
        command="uv",  # Executable
        args=[
            "run",
            "mcp",
            "run",
            "mcp_server.py",
        ],
    )

    # Client transport for stdio:
    # - Spawn a process which it will communicate with over stdin/stdout
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:

            # Initialize the connection
            await session.initialize()

            # Get tools in MCP server
            tools = await session.list_tools()

            # Format the tools for the agent
            mcp_tools = [tool.model_dump() for tool in tools.tools]

            # Fix json schema for input
            for tool in mcp_tools:
                tool["input_schema"] = tool["inputSchema"]
                del tool["inputSchema"]

            # Initialize the Anthropic client
            client = anthropic.Anthropic(api_key=API_KEY)

            # Create a message to send to the agent
            response = client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=1024,
                tools=mcp_tools,
                messages=[{"role": "user", "content": user_prompt}]
            )

            # Check if the agent's response includes a function call
            func_call = response.content[1].model_dump()

            # Call the function using the MCP server
            result = await session.call_tool(
                name=func_call["name"],
                arguments=func_call["input"]
            )

            word = func_call["input"]["word"]
            letter = func_call["input"]["letter"]
            count = result.content[0].text

            return f"There are {count} '{letter}'s in the word '{word}'."


In [9]:
await query_mcp_agent(user_prompt="How many R's are there in the word 'Strawberry'?")

"There are 3 'r's in the word 'Strawberry'."